<a href="https://colab.research.google.com/github/ryanch13579/Networking-Projects/blob/main/Network_Transmission_Visualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simple way to visual network data path captured from WireShark on Google Maps

Pre-requisites:
1. Download GeoliteCity
2. Get a pcap file from WireShark

In [ ]:
!pip install dpkt

In [ ]:
!pip install pygeoip

In [ ]:
import dpkt
import socket
import pygeoip

In [ ]:
gi = pygeoip.GeoIP('/content/Networking Project/Network Tracking using Wireshark/GeoLiteCity.dat')

In [ ]:
def retKML(dstrip, srcip):
  dst = gi.record_by_name(dstrip)
  src = gi.record_by_name('XXX.XXX.XXX.XXX') #Your personal Public IP
  try:
      dstlongitude = dst['longitude']
      dstlatitude = dst['latitude']
      srclongitude = src['longitude']
      srclatitude = src['latitude']
      kml = (
          '<Placemark>\n'
          '<name>%s</name>\n'
          '<extrude>1</extrude>\n'
          '<tessellate>1</tessellate>\n'
          '<styleUrl>#transBluePoly</styleUrl>\n'
          '<LineString>\n'
          '<coordinates>%6f,%6f\n%6f,%6f</coordinates>\n'
          '</LineString>\n'
          '</Placemark>\n'
      )%(dstrip, dstlongitude, dstlatitude, srclongitude, srclatitude)
      return kml
  except:
    return ''

In [ ]:
def plotIPs(pcap):
  kmlPts = ''
  for (ts, buf) in pcap:
    try:
      eth = dpkt.ethernet.Ethernet(buf)
      ip = eth.data
      src = socket.inet_ntoa(ip.src)
      dst = socket.inet_ntoa(ip.dst)
      KML = retKML(dst, src)
      kmlPts = kmlPts + KML
    except:
      pass
  return kmlPts

In [ ]:
def main():
  f = open('/content/Networking Project/Network Tracking using Wireshark/data.pcap', 'rb')
  pcap = dpkt.pcap.Reader(f)
  kmlheader = '<?xml version = "1.0" encoding = "UTF-8"?> \n<kml xmlns="http://www.opengis.net/kml/2.2">\n<Document>\n'\
              '<Style id="transBluePoly">' \
                '<LineStyle>' \
                '<width>1.5</width>' \
                '<color>501400E6</color>' \
                '</LineStyle>' \
                '</Style>'

  kmlfooter = '</Document>\n</kml>\n'
  kmldoc = kmlheader+plotIPs(pcap) + kmlfooter
  with open('/content/Networking Project/Network Tracking using Wireshark/output.kml', 'w') as kml_file:
    kml_file.write(kmldoc)
  print(kmldoc)

if __name__ == '__main__':
  main()
